# Flight Delay Project - Predictions
This Notebook is focused on predicting the Arrival Delay of flights.  This is difference between the actual time a flight arrives at the terminal and the time it was scheduled to arrive.

The predictions will take place once the plane takes off.

I will utilise 4 different machine learning models.

Linear Regression:
- Scikit-learn's ElasticNet

Gradient Boosting:
- LightGBM's LGBMRegressor
- XGBoost's XGBRegressor
- CatBoost's CatBoostRegressor

in order to handle the large amount of categorical data in landed_df.

# Step 1: Preprocessing the Data

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
landed_df = pd.read_pickle("landed_flights.pkl")

In [3]:
landed_df.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,SCHEDULED_DEPARTURE_HOURS,SCHEDULED_DEPARTURE_MINUTES,SCHEDULED_ARRIVAL_HOUR_IN_DESTINATION_TIMEZONE,SCHEDULED_ARRIVAL_MINUTE_IN_DESTINATION_TIMEZONE
0,1,1,4,AS,98,N407AS,ANC,SEA,2015-01-01 00:05:00,2014-12-31 23:54:00,...,-22,0,0,0,0,0,0,5,4,30
1,1,1,4,AA,2336,N3KUAA,LAX,PBI,2015-01-01 00:10:00,2015-01-01 00:02:00,...,-9,0,0,0,0,0,0,10,7,50
2,1,1,4,US,840,N171US,SFO,CLT,2015-01-01 00:20:00,2015-01-01 00:18:00,...,5,0,0,0,0,0,0,20,8,6
3,1,1,4,AA,258,N3HYAA,LAX,MIA,2015-01-01 00:20:00,2015-01-01 00:15:00,...,-9,0,0,0,0,0,0,20,8,5
4,1,1,4,AS,135,N527AS,SEA,ANC,2015-01-01 00:25:00,2015-01-01 00:24:00,...,-21,0,0,0,0,0,0,25,3,20


In [4]:
# Combine hours and minutes for Scheduled Departure.
landed_df["SCHEDULED_DEPARTURE_HOURS_MINUTES"] =\
    (landed_df["SCHEDULED_DEPARTURE_HOURS"].astype("uint16") * 60) + landed_df["SCHEDULED_DEPARTURE_MINUTES"]

# Combine hours and minutes for Scheduled Arrival.
landed_df["SCHEDULED_ARRIVAL_HOURS_MINUTES"] =\
    (landed_df["SCHEDULED_ARRIVAL"].dt.hour * 60) + landed_df["SCHEDULED_ARRIVAL"].dt.minute

# Combine hours and minutes for Scheduled Arrival in Destination Timezone.
landed_df["SCHEDULED_ARRIVAL_IN_DESTINATION_TIMEZONE_HOURS_MINUTES"] =\
    (landed_df["SCHEDULED_ARRIVAL_HOUR_IN_DESTINATION_TIMEZONE"].astype("uint16") * 60) + landed_df["SCHEDULED_ARRIVAL_MINUTE_IN_DESTINATION_TIMEZONE"]

# Drop the Hour columns, as they are replaced by the combined hour and minute columns as a record of the time of the day but keep the minute columns as categorical data.
landed_df = landed_df.drop(["SCHEDULED_DEPARTURE_HOURS","SCHEDULED_ARRIVAL_HOUR_IN_DESTINATION_TIMEZONE"], axis=1)

In [5]:
# Create a derivative feature for the Origin Airport and Airline combinations.
landed_df["OG_AIRPORT_AIRLINE"] = landed_df["ORIGIN_AIRPORT"].astype("str") + "__" + landed_df["AIRLINE"].astype("str")

# Change dtypes to category for categorical data
landed_df = landed_df.astype({"DAY_OF_WEEK":"category",
                              "MONTH":"category",
                              "DAY":"category",
                              "SCHEDULED_DEPARTURE_MINUTES":"category",
                              "SCHEDULED_ARRIVAL_MINUTE_IN_DESTINATION_TIMEZONE":"category",
                              "OG_AIRPORT_AIRLINE":"category"})

In [6]:
# Split my data into categorical and numerical columns.
cat_cols = ["AIRLINE",
            "ORIGIN_AIRPORT",
            "DESTINATION_AIRPORT",
            "DAY_OF_WEEK",
            "MONTH",
            "DAY",
            "SCHEDULED_DEPARTURE_MINUTES",
            "SCHEDULED_ARRIVAL_MINUTE_IN_DESTINATION_TIMEZONE",
            "OG_AIRPORT_AIRLINE"]

num_cols = ["DEPARTURE_DELAY",
            "TAXI_OUT",
            "DISTANCE",
            "SCHEDULED_DEPARTURE_HOURS_MINUTES",
            "SCHEDULED_ARRIVAL_HOURS_MINUTES",
            "SCHEDULED_ARRIVAL_IN_DESTINATION_TIMEZONE_HOURS_MINUTES"]

# I am transforming these 3 columns because they denote the time of day that a flight departed and I know from the Exploratory_Data_Analysis Notebook that the relationship between these columns and the Arrival Delay is non-linear.
transform_cols = ["SCHEDULED_DEPARTURE_HOURS_MINUTES",
                  "SCHEDULED_ARRIVAL_IN_DESTINATION_TIMEZONE_HOURS_MINUTES",
                  "SCHEDULED_ARRIVAL_HOURS_MINUTES"]

squared_data = landed_df[transform_cols].astype("float64") ** 2
squared_data.columns = [col + "_SQUARED" for col in transform_cols]

cubed_data = landed_df[transform_cols].astype("float64") ** 3
cubed_data.columns = [col + "_CUBED" for col in transform_cols]

# Adding the Transformed Data to landed_df.
landed_df = pd.concat([landed_df, squared_data, cubed_data], axis=1)

# Adding the transformed columns to num_cols.
num_cols = num_cols + squared_data.columns.to_list() + cubed_data.columns.to_list()

# Releasing memory.
squared_data, cubed_data = None, None
landed_df = landed_df.drop(["FLIGHT_NUMBER",
                            "TAIL_NUMBER",
                            "SCHEDULED_DEPARTURE",
                            "DEPARTURE_TIME",
                            "WHEELS_OFF",
                            "ELAPSED_TIME",
                            "AIR_TIME",
                            "TAXI_IN",
                            "WHEELS_ON",
                            "SCHEDULED_ARRIVAL",
                            "ARRIVAL_TIME"], axis=1)

# Creating a Dataframe for model comparison.  I will conduct a full model analysis at the end of the Notebook.
models = ["LinearRegression", "ElasticNet", "LGBMRegressor", "XGBRegressor", "CatBoostRegressor"]
model_comparison = pd.DataFrame({"Model": models,
                                 "Test_r2_score": 0,
                                 "Train_r2_score": 0,
                                 "Test_RMSE_score": 0,
                                 "Train_RMSE_score": 0}).set_index("Model")

In [7]:
# Convert categorical features to sparse one-hot encoded format for ElasticNet
from scipy import sparse

encoder = OneHotEncoder(sparse_output=True, drop="first")
cat_sparse = encoder.fit_transform(landed_df[cat_cols])

num_data = landed_df[num_cols]
scaler = StandardScaler(with_mean=False) # Keeping zeros to maintain sparsity and save memory
scaled_num_data = scaler.fit_transform(num_data)

num_sparse = sparse.csr_matrix(scaled_num_data)
data_sparse = sparse.hstack([cat_sparse, num_sparse])

***
# Step 2: Machine Learning

In [8]:
from sklearn.linear_model import LinearRegression

# Splitting the data.
train_sparse_data, test_sparse_data, train_result, test_result = train_test_split(data_sparse, landed_df["ARRIVAL_DELAY"], test_size=0.5, random_state=42)

# Defining the model.
linear =  LinearRegression()
linear.fit(train_sparse_data, train_result)

# Testing the model.
test_prediction = linear.predict(test_sparse_data)
train_prediction = linear.predict(train_sparse_data)

# Recording the accuracy of the model.
test_r2_score = r2_score(test_result, test_prediction)
train_r2_score = r2_score(train_result, train_prediction)
test_rmse_score = mean_squared_error(test_result, test_prediction, squared=False)
train_rmse_score = mean_squared_error(train_result, train_prediction, squared=False)
model_comparison.loc["LinearRegression"] = [test_r2_score, train_r2_score, test_rmse_score, train_rmse_score]

print("Test r2 Score:", test_r2_score)
print("Train r2 Score:", train_r2_score)
print("Test RMSE score:", test_rmse_score)
print("Train RMSE Score:", train_rmse_score)

Test r2 Score: 0.9391363054383819
Train r2 Score: 0.9393619233810927
Test RMSE score: 9.682145083281938
Train RMSE Score: 9.676767633158814


In [9]:
# Recovering the feature names from OneHotEncoder
cat_features = encoder.get_feature_names_out(cat_cols)

pd.DataFrame({"Feature": list(cat_features) + num_cols,
              "Weight": linear.coef_,
              "Abs_Weight":np.abs(linear.coef_)})\
    .set_index("Feature")\
    .nlargest(150, 'Abs_Weight')

,Weight,Abs_Weight
Feature,,
DEPARTURE_DELAY,36.773139,36.773139
OG_AIRPORT_AIRLINE_MSY__MQ,-15.262474,15.262474
OG_AIRPORT_AIRLINE_JFK__HA,15.224270,15.224270
ORIGIN_AIRPORT_LGA,-14.497642,14.497642
OG_AIRPORT_AIRLINE_CHS__AS,-14.272149,14.272149
...,...,...
OG_AIRPORT_AIRLINE_OTZ__AS,3.985705,3.985705
OG_AIRPORT_AIRLINE_RDU__AS,3.982987,3.982987
OG_AIRPORT_AIRLINE_LIH__UA,3.981480,3.981480


Departure Delay is the most important feature which we would expect based off of my data analysis.

We can also see that many airport airline combinations are part of the most important features as well as origin and destination airports.  This is also expected as they make up the bulk of the categorical features.

Only 2 airlines, Spirit, Frontier, and Hawaiian are included in the top 100 most important features.  Airlines despite having many more flights per airline than flights per -airline combination, are less important features (because there is less variation).

We can see the effectiveness of the airport-airline derivative feature right at the top of the Dataframe.  JFK as an Origin Airport has a weight of -11 but the JFK-Hawaiian combination has a weight of 8.  This features allows the model to understand the complex relationships between Airlines and Airports

***
# ElasticNet Linear Regression

Given that large size of the dataframe due to the categorical features that have been one-hot-encoded, I am using ElasticNet because it includes the cost of regularisation in its cost function and so aggressively reduces the dimensionality of the dataset by setting less important features to 0.  Based on my exploratory data analysis, I suspect that many of the airlines, and even the airports will have little importance on the arrival delay and so if ElasticNet identifies this and sets those features to 0 it should help avoid overfitting and reduce the time to compute.

### How Coordinate Descent in ElasticNet works:
ElasticNet takes the first w value (constant) for the first x value (feature) and optimises it with regards to a loss function.  The loss function incorporates both the Mean Squared Error (MSE) and both L1 and L2 regularisation, in order to both minimise the difference between the predicted value and the actual value, as well as the amount of regularisation that is applied to the w value.  The model doesn't use gradient descent, but coordinate descent instead (in order to include L1 regularisation in its loss function).  In coordinate descent, the model finds the optimal value that minimises the loss function and sets the w value to that optimal value.

Regularisation is used to prevent the model from overfitting (learning the training data so well that it can't effectively predict new data).

During optimisation, L1 regularisation adjusts the w value, each iteration, to reduce its size.  When the new w value is calculated by finding the importance of the feature in reducing the difference between the predicted value and the actual value, the feature may not be very important and so the w value may be below the threshold for L1 regularisation.  These w values are set to 0 because the feature's importance in reducing the loss function is less than the punishment, in the loss function, of using L1 regularisation on the feature.

For features, whose w values are above the threshold, they are pulled towards 0, each iteration by a fixed amount by L1 regularisation.  Then, they are pulled towards 0 by L2 regularisation which does so proportionately to the w values it is adjusting. This means that it prevents the model from relying too much on a few large features.

Once the model goes through every single w value, optimising each one, it goes back to the first w value and starts again.

This process is repeated until the cost function reaches a minimum or the maximum number of iterations has been reached.

ElasticNet can accept sparse data because it acts on each individual feature at a time and ignores "0" values.

In [10]:
from sklearn.linear_model import ElasticNet

# Defining my model, the param grid and the grid search.
elastic = ElasticNet()

param_grid = {"alpha":[0.005, 0.01, 0.02],
              "l1_ratio":[0.8, 0.9, 1.0]}

grid_search = GridSearchCV(elastic,
                           param_grid,
                           cv=3,
                           n_jobs=-1)

grid_search.fit(train_sparse_data, train_result)
best_elastic = grid_search.best_estimator_

# Testing the model.
test_prediction = best_elastic.predict(test_sparse_data)
train_prediction = best_elastic.predict(train_sparse_data)

# Recording the accuracy of the model.
test_r2_score = r2_score(test_result, test_prediction)
train_r2_score = r2_score(train_result, train_prediction)
test_rmse_score = mean_squared_error(test_result, test_prediction, squared=False)
train_rmse_score = mean_squared_error(train_result, train_prediction, squared=False)
model_comparison.loc["ElasticNet"] = [test_r2_score, train_r2_score, test_rmse_score, train_rmse_score]

print("Test r2 Score:", test_r2_score)
print("Train r2 Score:", train_r2_score)
print("Test RMSE score:", test_rmse_score)
print("Train RMSE Score:", train_rmse_score)
print(grid_search.best_params_)

Test r2 Score: 0.9375944698104983
Train r2 Score: 0.9377558793680788
Test RMSE score: 9.80401504383924
Train RMSE Score: 9.804078313326153
{'alpha': 0.005, 'l1_ratio': 1.0}


In [11]:
pd.DataFrame({"Feature": list(cat_features) + num_cols,
              "Weight": best_elastic.coef_,
              "Abs_Weight":np.abs(best_elastic.coef_)})\
    .set_index("Feature")\
    .nlargest(150, 'Abs_Weight')

,Weight,Abs_Weight
Feature,,
DEPARTURE_DELAY,36.775224,36.775224
ORIGIN_AIRPORT_LGA,-13.123136,13.123136
ORIGIN_AIRPORT_JFK,-12.580633,12.580633
ORIGIN_AIRPORT_EWR,-8.538402,8.538402
TAXI_OUT,8.387059,8.387059
...,...,...
DAY_6,-0.132796,0.132796
OG_AIRPORT_AIRLINE_MKE__WN,-0.116996,0.116996
DAY_14,-0.114286,0.114286


As we would expect from ElasticNet, the majority of features are set to 0.

Fewer Airport-Airline combinations are used than in Linear Regression.  This is because most Airport-Airline combinations have a low prevalence and so even though they are important when they do occur, they don't occur often enough to be considered important by Elastic Net.

We can see that many features such as the Day of the Week and the Months that were not in the top 150 for Linear Regression are given importance by Elastic Net.  This will be because they occur many times in the data and so impact more flights despite having less impact on those flights than other features that occur fewer times but have more impact.

***
# XGBRegressor Gradient Boost

I am using 3 different Gradient Boosting ML models.  Gradient Boosting, through its decision trees, can handle complex interactions between the categorical and numerical data.  In a situation where flights leaving JFK in the morning having a larger arrival delay than in the morning, when this is the opposite of the usual pattern seen in my data analysis, a Gradient Boosting model, through its decision trees would be able to handle this interaction which Linear Regression would miss.

### Creating the Decision Trees
The model first creates an initial prediction by predicting the mean probability of result_train (~0.53) for every single row. It does this by putting the log-odds of result_train and putting that into a sigmoid function. The model will be calculating log-odds

The model then finds the residuals (the difference between the predicted probability and the actual result) of each row. The residual is also called the gradient.

Then, the model selects a random subset of features (determined by colsample_by_tree), and evaluates every possible way to split the features and selects the split that best maximises the gain function.  To do this the split will:

- Maximises the square of the sum of residual values (to group together rows with similar errors).
- And minimises the sum of Hessian values (to prioritise correcting rows where the model is confidently incorrect).

This gives more weight to predicted probabilities that are very wrong. It increases the number of nodes (from 1 to 2).

The process repeats itself on each of the new nodes, splitting them as well, and continues to repeat until:

- The "max_depth" (number of successive splits) is reached.
- The sum of hessian values (of rows) at a node is less than "min_child_weight".
### Calculating the Probability
Once the tree has finished splitting the data, the model goes through each individual leaf and calculates for each row in that leaf a new probability.

It first calculates:

- The Hessian value: probability * (1 - probability)
- The log-odds: log( probability / (1 - probability) )

Calculate Delta/Leaf Score:

- All of the residual values in the leaf are added together.
- All of the hessian values in the leaf are added together.
- The sum of the residuals is then divided by: the sum of the hessians + L2 regularisation constant. The L2 regularisation constant is determined by "reg_lambda".
- This value is then multiplied by -1 (in order to minimise, rather than maximise, the loss function). This value is Delta, and it is stored in the leaf as the Leaf Score.

Calculate updated probability for each individual row:

- Delta is then multiplied by the "learning_rate".
- This is added to the log-odds for each individual row. (Each row has its own log-odds but the delta is shared by all rows in that leaf.)
- Then the new probability is calculated by putting the log-dds in the sigmoid function.
- Finally, the new residual value is calculated by finding the difference between the new probability and the actual result.

Then, the model takes the new residual values and moves onto the next tree. The number of trees is determined by "n_estimators".

Predicting the result
Each row is put through the first tree and follows the decision nodes until it makes it to a leaf. Once it is at a leaf, the Leaf Score of that leaf is added to the row's log-odds (which starts at 0).

Then each row moves on to the next tree and gets a Leaf Score added to its log-odds, and then the next tree and Leaf Score, and so on. When each row has passed through every single tree, the log-odds of each row is converted to probability through the sigmoid function. If the output of the sigmoid function is above 0.5 the model predicts, for that row, that the Blue team will win, if not, it predicts that the red team will win.

### My HyperParameters

I will explain some of my hyperparameters that I haven't touched on so far.

**tree_method="hist"**

This enables binning.  Binning is done natively by LGBMRegressor, which is the next Gradient Boosting model I will use but by setting tree_method to "hist" XGBRegressor can engage in Binning too.
- This puts features into bins which reduces the number of split points needed to be evaluated at each node.
- The Hessians and Gradients are calculated for all the values in each bin and so when a split point is evaluated the model doesn't need to add up the 5 million values again but just the 256 sums of the Hessians and Gradients for each bin.
- Bins are re-calculated at each individual node based off of the flights in that node.

**enable_categorical=True**

This allows the model to accept categorical data without one-hot encoding.

**n_jobs=-1**

This allows the model to use the maximum amount of computing power.

**min_child_weight=500**

This stops nodes from splitting and creating a leaf that would have less than 500 flights.  This helps to stop overfitting.  I have chosen a large number of 500 because my dataset is very large and my test size is over 2.5 million.

**n_estimators and max_depth**

I have chosen a relatively low list of potential n_estimators and a high list for max_depth.  This is because I think there will be many complicated interactions between the features (such as flights of a particular airport at a particular time, at a particular time of year, on a particular airline).  By having a large max_depth it increases the chances that the model can find and learn from these interactions.  The low n_estimators is in order to counteract the high max_depth and prevent overfitting.

In [12]:
from xgboost import XGBRegressor

# Splitting the data.  The 3 Gradient Boost algorithms I am using process categorical features natively (without the need to do one-hot encoding).
train_data, test_data, train_result, test_result = train_test_split(landed_df[cat_cols + num_cols], landed_df["ARRIVAL_DELAY"], test_size=0.5, random_state=42)

# Defining my model, the param grid and the grid search.
xreg = XGBRegressor(tree_method="hist",
                    enable_categorical=True,
                    reg_alpha=1.5,
                    reg_lambda=1.5,
                    colsample_bytree=0.5,
                    n_jobs=-1,
                    min_child_weight=500)

param_grid = {"n_estimators": [60, 80, 100, 120],
              "max_depth": [40, 50, 60],
              "min_child_weight":[500, 750, 1000],
              "learning_rate": [0.1, 0.2, 0.3]}

grid_search = GridSearchCV(xreg,
                           param_grid,
                           cv=3,
                           n_jobs=-1)

grid_search.fit(train_data, train_result)
best_xreg = grid_search.best_estimator_

# Testing the model.
test_prediction = best_xreg.predict(test_data)
train_prediction = best_xreg.predict(train_data)

# Recording the accuracy of the model.
test_r2_score = r2_score(test_result, test_prediction)
train_r2_score = r2_score(train_result, train_prediction)
test_rmse_score = mean_squared_error(test_result, test_prediction, squared=False)
train_rmse_score = mean_squared_error(train_result, train_prediction, squared=False)
model_comparison.loc["XGBRegressor"] = [test_r2_score, train_r2_score, test_rmse_score, train_rmse_score]

print("Test r2 Score:", test_r2_score)
print("Train r2 Score:", train_r2_score)
print("Test RMSE score:", test_rmse_score)
print("Train RMSE Score:", train_rmse_score)
print(grid_search.best_params_)

Test r2 Score: 0.901951307989357
Train r2 Score: 0.9251044005815546
Test RMSE score: 12.28891
Train RMSE Score: 10.754391
{'learning_rate': 0.2, 'max_depth': 40, 'min_child_weight': 500, 'n_estimators': 120}


In [13]:
{'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 200}

{'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 200}

In [14]:
xreg_importance = (pd.Series(best_xreg.get_booster().get_score(importance_type="gain"),
                             name="Importance")
                   .sort_values(ascending=False))

xreg_importance/xreg_importance.sum()

DEPARTURE_DELAY                                                    0.858357
TAXI_OUT                                                           0.054472
OG_AIRPORT_AIRLINE                                                 0.018379
AIRLINE                                                            0.010809
DESTINATION_AIRPORT                                                0.007590
ORIGIN_AIRPORT                                                     0.007380
MONTH                                                              0.005810
SCHEDULED_DEPARTURE_HOURS_MINUTES                                  0.004279
DAY                                                                0.004198
DAY_OF_WEEK                                                        0.003478
DISTANCE                                                           0.003339
SCHEDULED_DEPARTURE_HOURS_MINUTES_SQUARED                          0.002618
SCHEDULED_ARRIVAL_IN_DESTINATION_TIMEZONE_HOURS_MINUTES_SQUARED    0.002578
SCHEDULED_AR

Departure Delay, as expected is the most important feature followed by Taxi Out.

The least important feature is the day of the week.

The Airport Airline combination is the most important categorical feature.

***
# LGBMRegressor Gradient Boost

I am using LGBMRegressor because it has a fast time to compute and is better at dealing with large numbers of categorical features.

It is similar to XGBRegressor and has the same Gain function (to decide how to split each node), but it works differently and should be more suitable for this project.

This is due to several reasons, but I will describe just 2 of them:

**Binning Strategy**
- LGBMRegressor sorts the categories by the size of their Gradient to Hessian ratio in order to find categories that are most similar to each other to bin together.

**Leaf-wise Growth**
- LGBMRegressor also uses Leaf-wise Tree growth.  This is different to XGBRegressor, which splits every leaf on each level before moving onto the next one.
- LGBMRegressor does not split every single leaf on a level before moving onto the next leaf and will evaluate every single leaf (on all levels) to decide which is the best split.  This means that it makes fewer low-Gain splits because it prioritises the leaves that maximise the Gain function rather than just prioritising the splits that do so and will reach its max_depth by splitting fewer leaves because it enacts fewer low-Gain splits.

LGBMRegressor is, in general, just better at handling large amounts of categorical data than XBGRegressor.

### Parameters

I am using large values for "num_leaves" in order to increase the chances of the model finding complicated interactions between features in the data.  In my preliminary run of the model, I did not have any significant problems with overfitting, and so I have not needed to use parameters such as max_depth, min_data_in_leaf, feature_fraction, lambda_l1, or lambda_l2.  However, I have set min_data_in_leaf to its default value.

In [33]:
from lightgbm import LGBMRegressor

# Defining my model, the param grid and the grid search.
lreg = LGBMRegressor(objective="regression",
                     random_state=42,
                     verbose=-1,
                     min_data_in_leaf=20)

param_grid = {"n_estimators": [100, 150, 200, 250],
              "num_leaves": [300, 350, 400, 450, 500],
              "learning_rate": [0.1, 0.2, 0.4, 0.8]}

grid_search = GridSearchCV(lreg,
                           param_grid,
                           cv=3,
                           n_jobs=-1)

grid_search.fit(train_data, train_result)
best_lreg = grid_search.best_estimator_

# Testing the model.
test_prediction = best_lreg.predict(test_data)
train_prediction = best_lreg.predict(train_data)

# Recording the accuracy of the model.
test_r2_score = r2_score(test_result, test_prediction)
train_r2_score = r2_score(train_result, train_prediction)
test_rmse_score = mean_squared_error(test_result, test_prediction, squared=False)
train_rmse_score = mean_squared_error(train_result, train_prediction, squared=False)
model_comparison.loc["LGBMRegressor"] = [test_r2_score, train_r2_score, test_rmse_score, train_rmse_score]

print("Test r2 Score:", test_r2_score)
print("Train r2 Score:", train_r2_score)
print("Test RMSE score:", test_rmse_score)
print("Train RMSE Score:", train_rmse_score)
print(grid_search.best_params_)

Test r2 Score: 0.9527491693476293
Train r2 Score: 0.9676530141774808
Test RMSE score: 8.530945239022634
Train RMSE Score: 7.0676415116117814
{'learning_rate': 0.2, 'n_estimators': 250, 'num_leaves': 350}


In [25]:
# Feature Importance
lreg_importance = (pd.Series(best_lreg.booster_.feature_importance(importance_type='gain'),
                             index=cat_cols + num_cols,
                             name="Importance")
                   .sort_values(ascending=False))

lreg_importance / lreg_importance.sum()

DEPARTURE_DELAY                                                    0.918609
TAXI_OUT                                                           0.035549
OG_AIRPORT_AIRLINE                                                 0.016885
DESTINATION_AIRPORT                                                0.006198
ORIGIN_AIRPORT                                                     0.005465
DAY                                                                0.003450
SCHEDULED_ARRIVAL_MINUTE_IN_DESTINATION_TIMEZONE                   0.002824
DISTANCE                                                           0.002496
MONTH                                                              0.002332
AIRLINE                                                            0.001638
SCHEDULED_DEPARTURE_MINUTES                                        0.001056
SCHEDULED_ARRIVAL_IN_DESTINATION_TIMEZONE_HOURS_MINUTES            0.000999
SCHEDULED_DEPARTURE_HOURS_MINUTES                                  0.000979
SCHEDULED_AR

In [26]:
# Feature Importance Analysis Here

# CatBoostRegressor Gradient Boost

I am using CatBoostRegressor as a 3rd Gradient Boosting model in order to compare to LGBRegressor and XGBRegressor.  However, I do not think it will produce better results that LGBRegressor, I would still like to compare the results and feature importance of the models.

The reason why I don't think it will have a higher predicted accuracy than LGBRegressor is because the patterns in the data are highly specialised and may be unique for individual airports and airlines.  This is why I set no max_depth for LGBRegressor.

When CatBoostRegressor is deciding how to split its nodes, it does not do, as XBGRegressor and LGBRegressor do and split each node individually.  It splits every single node (on the same level) the exact same way, so when it is evaluating a split, it evaluates the Gain function for each individual node and then sums the Gain functions all together.  This prioritises finding patterns that are present across the dataset rather than for the samples in an individual node which both helps prevent overfitting and can cause the model to miss out on real interactions within the data.

In [27]:
from catboost import CatBoostRegressor

# Defining my model, the param grid and the grid search.
creg = CatBoostRegressor(random_state=42,
                         thread_count=-1,
                         verbose=False)

param_grid = {"depth": [10, 12, 14, 16],
              "iterations": [60, 80, 100, 120, 140],
              "learning_rate":[0.1, 0.2, 0.3]}

grid_search = GridSearchCV(creg,
                           param_grid,
                           cv=2,
                           n_jobs=-1)

grid_search.fit(train_data, train_result, cat_features=cat_cols)
best_creg = grid_search.best_estimator_

# Testing the model.
test_prediction = best_creg.predict(test_data)
train_prediction = best_creg.predict(train_data)

# Recording the accuracy of the model.
test_r2_score = r2_score(test_result, test_prediction)
train_r2_score = r2_score(train_result, train_prediction)
test_rmse_score = mean_squared_error(test_result, test_prediction, squared=False)
train_rmse_score = mean_squared_error(train_result, train_prediction, squared=False)
model_comparison.loc["CatBoostRegressor"] = [test_r2_score, train_r2_score, test_rmse_score, train_rmse_score]

print("Test r2 Score:", test_r2_score)
print("Train r2 Score:", train_r2_score)
print("Test RMSE score:", test_rmse_score)
print("Train RMSE Score:", train_rmse_score)

print(grid_search.best_params_)

C:\Users\busin\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
22 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
22 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\busin\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\busin\miniconda3\lib\site-packages\catboost\core.py", line 5873, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline,
  File "C:\Users\busin\miniconda3\lib\site-packages\catboost\core.py", line 2410

Test r2 Score: 0.9421687451528881
Train r2 Score: 0.9475266386685753
Test RMSE score: 9.437864509938322
Train RMSE Score: 9.001752051097325
{'depth': 14, 'iterations': 140, 'learning_rate': 0.3}


In [28]:
creg_importance = (pd.Series(best_creg.get_feature_importance(),
                            index=cat_cols + num_cols,
                            name="Importance")
                   .sort_values(ascending=False))

creg_importance / creg_importance.sum()

DEPARTURE_DELAY                                                    0.596280
TAXI_OUT                                                           0.116930
ORIGIN_AIRPORT                                                     0.062287
AIRLINE                                                            0.047204
DISTANCE                                                           0.044028
DESTINATION_AIRPORT                                                0.025829
OG_AIRPORT_AIRLINE                                                 0.021349
MONTH                                                              0.019464
SCHEDULED_DEPARTURE_HOURS_MINUTES_CUBED                            0.007974
SCHEDULED_ARRIVAL_HOURS_MINUTES                                    0.006893
SCHEDULED_DEPARTURE_HOURS_MINUTES                                  0.006665
SCHEDULED_ARRIVAL_HOURS_MINUTES_SQUARED                            0.006612
SCHEDULED_DEPARTURE_MINUTES                                        0.006486
SCHEDULED_DE

# Stacking Model

In [29]:
meta_model_train_data = np.column_stack([best_lreg.predict(train_data),
                                         linear.predict(train_sparse_data)])


stacking_model = LinearRegression()

stacking_model.fit(meta_model_train_data, train_result)

meta_model_test_data = np.column_stack([best_lreg.predict(test_data),
                                        linear.predict(test_sparse_data)])


test_prediction = stacking_model.predict(meta_model_test_data)
train_prediction = stacking_model.predict(meta_model_train_data)

test_r2_score = r2_score(test_result, test_prediction)
train_r2_score = r2_score(train_result, train_prediction)
test_rmse_score = mean_squared_error(test_result, test_prediction, squared=False)
train_rmse_score = mean_squared_error(train_result, train_prediction, squared=False)
model_comparison.loc["Stacking_Model"] = [test_r2_score, train_r2_score, test_rmse_score, train_rmse_score]

print("Test r2 Score:", test_r2_score)
print("Train r2 Score:", train_r2_score)
print("Test RMSE score:", test_rmse_score)
print("Train RMSE Score:", train_rmse_score)

Test r2 Score: 0.9496712185106274
Train r2 Score: 0.9705395504341796
Test RMSE score: 8.804417701209655
Train RMSE Score: 6.744927544466131


In [30]:
pd.DataFrame({"Base_Model":["LGBMRegressor", "Linear Regression"],
              "Weight":stacking_model.coef_})

,Base_Model,Weight
0,LGBMRegressor,1.263697
1,Linear Regression,-0.263340


# Model Analysis

In [31]:
model_comparison.sort_values(by="Test_r2_score", ascending=False)

,Test_r2_score,Train_r2_score,Test_RMSE_score,Train_RMSE_score
Model,,,,
LGBMRegressor,0.952523,0.969182,8.551310,6.898595
Stacking_Model,0.949671,0.970540,8.804418,6.744928
CatBoostRegressor,0.942169,0.947527,9.437865,9.001752
LinearRegression,0.939136,0.939362,9.682145,9.676768
ElasticNet,0.937594,0.937756,9.804015,9.804078
XGBRegressor,0.901951,0.925104,12.288910,10.754391


In [32]:
landed_df["ARRIVAL_DELAY"].std()

39.271297093886396

# Conclusions